In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from PIL import Image
import glob as glob
import cv2
import sys
from os import listdir
from os.path import isfile, join
import time
from PIL import ImageFile
import shutil
%matplotlib inline
sys.path.append("..")
CWD_PATH = os.getcwd()

2022-01-27 04:24:03.336671: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
from utils import label_map_util
from utils import visualization_utils as vis_util

In [3]:
MODEL_NAME = 'ssd_mobilenet_v2'

In [4]:
MODEL_PATH = os.path.join('model', MODEL_NAME)
PATH_TO_CKPT = os.path.join(MODEL_PATH,'weights','saved_model')
#PATH_TO_CKPT = os.path.join(MODEL_PATH,'saved_model')
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('fruit_detection','data', 'fruit.pbtxt')

NUM_CLASSES = 3

In [5]:
print('PATH_TO_CKPT :', PATH_TO_CKPT)
print('PATH_TO_LABELS :', PATH_TO_LABELS)
tf.keras.backend.clear_session()
detect_fn = tf.saved_model.load(PATH_TO_CKPT)

PATH_TO_CKPT : model/ssd_mobilenet_v2/weights/saved_model
PATH_TO_LABELS : fruit_detection/data/fruit.pbtxt


2022-01-27 04:25:46.464576: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-27 04:25:46.465117: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-27 04:25:46.518450: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:0a:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.6705GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-01-27 04:25:46.518916: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:41:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.6705GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2022-01-27 04:25:46.518943: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda

In [6]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
print(label_map)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
print(categories)
category_index = label_map_util.create_category_index(categories)
print(category_index)

item {
  name: "orange"
  id: 1
}
item {
  name: "apple"
  id: 2
}
item {
  name: "banana"
  id: 3
}

[{'id': 1, 'name': 'orange'}, {'id': 2, 'name': 'apple'}, {'id': 3, 'name': 'banana'}]
{1: {'id': 1, 'name': 'orange'}, 2: {'id': 2, 'name': 'apple'}, 3: {'id': 3, 'name': 'banana'}}


In [7]:
PATH_TO_TEST_IMAGES_DIR = 'fruit_detection/fruit_data/test'
image_name=[]

In [8]:
from natsort import natsorted
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, f) for f in listdir(PATH_TO_TEST_IMAGES_DIR) if isfile(os.path.join(PATH_TO_TEST_IMAGES_DIR, f))]
TEST_IMAGE_PATHS = sorted(TEST_IMAGE_PATHS)
TEST_IMAGE_PATHS = natsorted(TEST_IMAGE_PATHS)
#print('TEST_IMAGE_PATHS :',TEST_IMAGE_PATHS)

In [9]:
image_name=[f for f in listdir(PATH_TO_TEST_IMAGES_DIR) if isfile(os.path.join(PATH_TO_TEST_IMAGES_DIR, f))]
image_name = sorted(image_name)
image_name = natsorted(image_name)
print(image_name)

['apple_77.jpg', 'apple_77.xml', 'apple_78.jpg', 'apple_78.xml', 'apple_79.jpg', 'apple_79.xml', 'apple_80.jpg', 'apple_80.xml', 'apple_81.jpg', 'apple_81.xml', 'apple_82.jpg', 'apple_82.xml', 'apple_83.jpg', 'apple_83.xml', 'apple_84.jpg', 'apple_84.xml', 'apple_85.jpg', 'apple_85.xml', 'apple_86.jpg', 'apple_86.xml', 'apple_87.jpg', 'apple_87.xml', 'apple_88.jpg', 'apple_88.xml', 'apple_89.jpg', 'apple_89.xml', 'banana_77.jpg', 'banana_77.xml', 'banana_78.jpg', 'banana_78.xml', 'banana_79.jpg', 'banana_79.xml', 'banana_80.jpg', 'banana_80.xml', 'banana_81.jpg', 'banana_81.xml', 'banana_82.jpg', 'banana_82.xml', 'banana_83.jpg', 'banana_83.xml', 'banana_84.jpg', 'banana_84.xml', 'banana_85.jpg', 'banana_85.xml', 'banana_86.jpg', 'banana_86.xml', 'banana_87.jpg', 'banana_87.xml', 'banana_88.jpg', 'banana_88.xml', 'banana_89.jpg', 'banana_89.xml', 'mixed_21.jpg', 'mixed_21.xml', 'mixed_22.jpg', 'mixed_22.xml', 'mixed_23.jpg', 'mixed_23.xml', 'orange_77.jpg', 'orange_77.xml', 'orange_78.

In [10]:
#def DetectImagesFromFolder(detector, images_dir, save_output=False, output_dir='output/'):

for file in os.scandir(PATH_TO_TEST_IMAGES_DIR):
    if file.is_file() and file.name.endswith(('.jpg', '.jpeg', '.png')) :
        image_path = os.path.join(PATH_TO_TEST_IMAGES_DIR, file.name)
        #class_id = 1
        print(image_path)
        img = cv2.imread(image_path)
        image_np1 = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        input_tensor = np.expand_dims(img, 0)
        detections = detect_fn(input_tensor)
        bboxes = detections['detection_boxes'][0].numpy()
        bclasses = detections['detection_classes'][0].numpy().astype(np.int32)
        bscores = detections['detection_scores'][0].numpy()
        IMAGE_SIZE = (20, 20)
        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np1,
            np.squeeze(bboxes),
            np.squeeze(bclasses).astype(np.int32),
            np.squeeze(bscores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=4,
            min_score_thresh=0.4)
        im = Image.fromarray(image_np1)
        im.save(os.path.join(CWD_PATH, 'output_fruit',file.name))

fruit_detection/fruit_data/test/orange_83.jpg


2022-01-27 04:29:11.857921: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-27 04:29:11.947478: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3399685000 Hz
2022-01-27 04:29:12.457582: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2022-01-27 04:29:13.329103: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-01-27 04:29:13.329729: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11


fruit_detection/fruit_data/test/apple_78.jpg
fruit_detection/fruit_data/test/apple_84.jpg
fruit_detection/fruit_data/test/orange_86.jpg
fruit_detection/fruit_data/test/orange_89.jpg
fruit_detection/fruit_data/test/banana_80.jpg
fruit_detection/fruit_data/test/apple_85.jpg
fruit_detection/fruit_data/test/banana_84.jpg
fruit_detection/fruit_data/test/mixed_22.jpg
fruit_detection/fruit_data/test/apple_82.jpg
fruit_detection/fruit_data/test/banana_86.jpg
fruit_detection/fruit_data/test/orange_79.jpg
fruit_detection/fruit_data/test/apple_81.jpg
fruit_detection/fruit_data/test/orange_87.jpg
fruit_detection/fruit_data/test/banana_78.jpg
fruit_detection/fruit_data/test/apple_77.jpg
fruit_detection/fruit_data/test/banana_85.jpg
fruit_detection/fruit_data/test/orange_81.jpg
fruit_detection/fruit_data/test/banana_87.jpg
fruit_detection/fruit_data/test/apple_87.jpg
fruit_detection/fruit_data/test/banana_81.jpg
fruit_detection/fruit_data/test/mixed_23.jpg
fruit_detection/fruit_data/test/orange_78.j